In [4]:
#Get Neo4J official Python driver
!pip install neo4j

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
Could not import setuptools which is required to install from a source distribution.
Please install setuptools.
You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
def connect(uri, username, password):
    try:
        return GraphDatabase.driver(uri, auth=(username, password))
    except ClientError as e:
        print(e)
        return -1

In [ ]:
def create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

In [ ]:
def add_user(tx, username, password):
    try:
        result = tx.run("CALL dbms.security.createUser($username,$password, false)", username=username, password=password)
    except ClientError as e:
        print(e)

In [ ]:
def change_password(tx, password):
    try:
        result = tx.run("CALL dbms.security.changePassword($password)", password=password)
    except ClientError as e:
        print(e)

In [ ]:
#Neo4J connection test
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

uri = "bolt://localhost:7687"
our_username = "ads"
our_password = "ads"
message = "Response"
default = "neo4j"

# Logging with default user and password
driver = connect(uri, default, default)
# If we have already run this before, then the password is already changed
if driver != -1:
    ''' But if it is the first time we log as that default user, 
        we are forced to change the password '''
    with driver.session() as session:
        session.write_transaction(change_password, our_password) # e.g. our_password
    # We should exit and connect again after changing the password
    driver.close()

driver = connect(uri, default, our_password)

# Add the user that will be used for the rest of operations
with driver.session() as session:
    try: 
        session.write_transaction(add_user, our_username, our_password)
    except ClientError as e:
        print(e)
driver.close()

driver = connect(uri, our_username, our_password)
with driver.session() as session:
    greeting = session.write_transaction(create_and_return_greeting, message)
    print(greeting) 
driver.close()

In [3]:
#Get Requests
!pip3 install beautifulsoup4

  Using cached https://files.pythonhosted.org/packages/1d/5d/3260694a59df0ec52f8b4883f5d23b130bc237602a1411fa670eae12351e/beautifulsoup4-4.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/a5/7ea40d0f8676bde6e464a6435a48bc5db09b1a8f4f06d41dd997b8f3c616/soupsieve-1.9.1-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
#Simple get example
import requests

r = requests.get('https://www.agh.edu.pl/')
r.status_code
r.headers['content-type']
r.encoding
# r.text holds html file

'utf-8'

In [21]:
#Crawler workflow
import requests, re
from bs4 import BeautifulSoup

queue_limit = 10
visited = set()
to_visit = ['https://www.agh.edu.pl/']
crawled_host = 'https://www.agh.edu.pl'

results = []

while len(to_visit) > 0 and len(results) < queue_limit :

    crawled_url = to_visit.pop()
    urls = []

    if crawled_url not in visited:
        r = requests.get(crawled_url)
        visited.add(crawled_url)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text)
            
            for a in soup.find_all('a', href=True):
                current_path = a['href']
            
                if current_path.startswith('/') and not '#' in current_path:
                    current_url = crawled_host + current_path
                
                    if current_url not in visited:
                        urls.append(current_url)
                        to_visit.append(current_url)
                        # print("Found the URL:", current_url)
            
            #TODO find also meta_data
            result = (crawled_url, urls, {"metadata" : "TODO"})
            results.append(result)
        else:
            print(r.status_code)
    
print(results)
print(visited)
print(to_visit)

1
2
3
4
5
6
7
8
9
10
[('https://www.agh.edu.pl/', ['https://www.agh.edu.pl/mapa-strony/', 'https://www.agh.edu.pl/en/', 'https://www.agh.edu.pl/kandydaci/', 'https://www.agh.edu.pl/studenci/', 'https://www.agh.edu.pl/doktoranci/', 'https://www.agh.edu.pl/pracownicy/', 'https://www.agh.edu.pl/absolwenci/', 'https://www.agh.edu.pl/info/article/dzien-agh-8-czerwca/', 'https://www.agh.edu.pl/info/article/casting-do-widowiska-teatralno-muzycznego-z-okazji-jubileuszu-100-lecia-agh/', 'https://www.agh.edu.pl/info/article/rozrywkowo-z-con-fuoco-1/', 'https://www.agh.edu.pl/info/article/komputer-jako-pomoc-dla-osob-piszacych-felieton/', 'https://www.agh.edu.pl/aktualnosci/', 'https://www.agh.edu.pl/osiagniecia/', 'https://www.agh.edu.pl/nauka/', 'https://www.agh.edu.pl/uczelnia/multimedia/', 'https://www.agh.edu.pl/wydarzenia/', 'https://www.agh.edu.pl/konferencje/', 'https://www.agh.edu.pl/wydarzenia/kultura/', 'https://www.agh.edu.pl/wydarzenia/info/article/61-wystawa-i-gielda-mineralow-skami